# List of My Movies

In [1]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from combine import combine


import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.6.6 |Anaconda custom (64-bit)| (default, Jun 28 2018, 11:07:29) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2018-12-30 18:18:30.029405


In [2]:
import re
from time import sleep
from collections import OrderedDict
from timeUtils import clock, elapsed
from ioUtils import saveFile, getFile
from fsUtils import setDir, isDir, mkDir, setFile, isFile, setSubFile
from fileUtils import getBaseFilename
from searchUtils import findSubPatternExt, findPatternExt, findExt, findNearest
from strUtils import convertCurrency
from webUtils import getWebData, getHTML
from numpy import repeat
from movieDB import movieDB
from os import getcwd
from glob import glob
import operator
from movieRenames import manualRenames

##############################################################################################################################
# Combine Movies
##############################################################################################################################
class mymovies(movieDB):
    def __init__(self, basedir=None):
        self.name = "mymovies"
        movieDB.__init__(self, dbdir=self.name)
        self.combine = None
        
    def setCombinedMovies(self, combine):
        self.combine = combine
        
    def getCombinedMovies(self, debug=False):        
        savename = setFile(self.combine.getResultsDir(), "movies.yaml")
        if not isFile(savename):
            raise ValueErrro("Cannot access {0}".format(savename))
        combinedMovies = getFile(savename)
        if debug:
            print("Found {0} combined movies".format(len(combinedMovies)))
        return combinedMovies
        
    def getMyMovies(self, debug=False): 
        savename = setFile(self.getDataDir(), "mymovies.json")
        if not isFile(savename):
            raise ValueErrro("Cannot access {0}".format(savename))
        mine = getFile(savename)
        if debug:
            print("Found {0} my movies".format(len(mine)))
        return mine
        
    def findMyMovies(self, debug=False):
        movies = glob("/Volumes/*/Movies/*")
        mine   = dict(zip([getBaseFilename(x) for x in movies], movies))
        print("Found {0} movies on my disks".format(len(movies)))
        savename = setFile(self.getDataDir(), "mymovies.json")
        saveFile(idata=mine, ifile=savename, debug=True)
        
        
    def searchMyMovies(self, movie, year, cutoff=0.8):
        moviesMine = self.getMyMovies()
        key = "{0} [{1}]".format(movie, year)
        result     = findNearest(key, moviesMine.keys(), 1, 0.95)
        if len(result) > 0:
            print("{0}  --->  {1}".format(movie, result))
            return
        else:
            key = "{0} [{1}]".format(movie, year)
            result = findNearest(key, moviesMine.keys(), 1, cutoff)
            if len(result) > 0:
                print("{0}  --->  {1}".format(movie, result))
                return
        print("No match found")
        return
        
        
        
    def mergeMovies(self, toFile=False):
        moviesCombined = self.getCombinedMovies()
        moviesMine     = self.getMyMovies()
        filename = setFile(self.getResultsDir(), "mymovies.dat")

        if toFile:
            print("{0: <6}{1: <3}{2: <100}{3: <20}".format("Year", u'\u2713', "Movie", "Source"), file=open(filename, "w"))
        else:
            print("{0: <6}{1: <3}{2: <100}{3: <20}".format("Year", u'\u2713', "Movie", "Source"))

            
        for key,name in moviesCombined.items():
            movie = key[:-7]
            year  = key[-5:-1]

            location = moviesMine.get(key)
            if location is None:
                location = ""
            else:
                location = u'\u2713'
                
            if toFile:
                print("{0: <6}{1: <3}{2: <100}{3: <20}".format(year, location, movie, name), file=open(filename, "a"))
            else:
                print("{0: <6}{1: <3}{2: <100}{3: <20}".format(year, location, movie, name))

In [3]:
mine = mymovies()
mine.setCombinedMovies(combine())
mine.findMyMovies()
#moviesMine     = mine.getMyMovies()
#moviesCombined = mine.getCombinedMovies()

Found 4866 movies on my disks
Saving data to /Users/tgadfort/Documents/code/movies/mymovies/data/mymovies.json
Saved data to /Users/tgadfort/Documents/code/movies/mymovies/data/mymovies.json
  --> This file is 395.3kB.


In [4]:
mine.searchMyMovies("Beerfest", 2011)

Beerfest  --->  ['Beerfest [2006]']


In [5]:
mine.mergeMovies()

Year  ✓  Movie                                                                                               Source              
1915     The Birth Of A Nation                                                                               Rotten Tomatoes     
1920     The Cabinet Of Dr. Caligari (Das Cabinet des Dr. Caligari)                                          Rotten Tomatoes     
1922     Nosferatu, A Symphony Of Horror (Nosferatu, eine Symphonie des Grauens) (Nosferatu The Vampire)     Rotten Tomatoes     
1925     The Gold Rush                                                                                       Rotten Tomatoes     
1925     Battleship Potemkin                                                                                 Rotten Tomatoes     
1925     Phantom Of The Opera                                                                                Rotten Tomatoes     
1927     Metropolis                                                                       

1992  ✓  Stop! Or My Mom Will Shoot                                                                          Box Office          
1992     Candyman                                                                                            Box Office          
1992     Consenting Adults                                                                                   Box Office          
1992  ✓  Ferngully The Last Rainforest                                                                       Box Office          
1992  ✓  Raising Cain                                                                                        Box Office          
1992     Shining Through                                                                                     Box Office          
1992     The Cutting Edge                                                                                    Box Office          
1992  ✓  Forever Young                                                                    

1999  ✓  Varsity Blues                                                                                       Box Office          
1999  ✓  Message In A Bottle                                                                                 Box Office          
1999  ✓  South Park Bigger, Longer And Uncut                                                                 Box Office          
1999     Mighty Joe Young                                                                                    Box Office          
1999  ✓  Stigmata                                                                                            Box Office          
1999     Practical Magic                                                                                     Box Office          
1999     Meet Joe Black                                                                                      Box Office          
1999     The Siege                                                                        

2006  ✓  16 Blocks                                                                                           Box Office          
2006  ✓  Accepted                                                                                            Box Office          
2006  ✓  The Sentinel                                                                                        Box Office          
2006     The Nativity Story                                                                                  Box Office          
2006     Derailed                                                                                            Box Office          
2006  ✓  Blood Diamond                                                                                       Box Office          
2006     The Ringer                                                                                          Box Office          
2006  ✓  The Good Shepherd                                                                

2013     War Witch                                                                                           Rotten Tomatoes     
2013     Still Mine                                                                                          Rotten Tomatoes     
2013     Drug War                                                                                            Rotten Tomatoes     
2013     A Band Called Death                                                                                 Rotten Tomatoes     
2013  ✓  Inside Llewyn Davis                                                                                 Rotten Tomatoes     
2013  ✓  Captain Phillips                                                                                    Rotten Tomatoes     
2013     After Tiller                                                                                        Rotten Tomatoes     
2013     Our Children                                                                     